First import modules and initialize Earth Engine.

In [1]:

# standard modules
import io
import json
import os
from pathlib import Path
import time

# specialized modules
import ee
import geemap
import geopandas as gpd
from pathlib import Path
from tqdm import tqdm

# initialize the Earth Engine module.
ee.Initialize(project='trinity-438000')

/home/michael/miniforge3/envs/ee/lib/python3.13/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Open the AOI vector file and check it on a map.

In [2]:
# read AOI
dot_slash = Path().cwd()
aoi_path = dot_slash.parent / 'untracked_qgis' / 'aoi.geojson'


In [3]:
aoi = gpd.read_file(aoi_path).to_crs(4326)
gee_json = json.loads(aoi[['geometry']].to_json())
gee_aoi = geemap.geojson_to_ee(gee_json)

# inspect the GeoJSON as an EEObject through geemap.
test_map = geemap.Map()
test_map.centerObject(gee_aoi, 9) # Centering on the AOI instead of an undefined 'region'
test_map.addLayer(gee_aoi, {}, 'AOI')

test_map


EEException: Invalid JSON payload received. Unexpected token.
 {"constantValue": [Infinity, Infinity]}
                    ^

Get the vertices of the extent of the AOI to use in queries to API.

In [ ]:
# get extent
minx, miny, maxx, maxy = aoi.total_bounds

verts = [[
[minx, miny],
[minx, maxy],
[maxx, maxy],
[maxx, miny],
[minx, miny]
]]

# make normal float from np.float63
verts = [[float(x), float(y)] for x, y in verts[0]]

verts

NameError: name 'aoi' is not defined

We will now download Harmonized Sentinel-2 MSI: MultiSpectral Instrument, Level-2A (SR), 



In [ ]:
# date range
START = ee.Date('2015-01-01')
END = ee.Date('2025-08-01')

# date and geographic filter
col_filter = ee.Filter.And(
    ee.Filter.geometry(ee.Geometry.Polygon(verts)),
    ee.Filter.date(START, END),
)

# collections of interest
s2_col = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filter(col_filter)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35))
)

In [ ]:
def mask_s2_clouds(image):
  '''Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  '''
  qa = image.select('QA60')

  # bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [ ]:
# dowland dirs
s2_dir = Path('/home/michael/OneDrive/nr218_files/s2')


tasks = []
for year in tqdm(range(2016, 2026)):
    # filter for June and AOI
    start = ee.Date(f'{year}-06-01')
    end = ee.Date(f'{year}-06-31')
    lil_filter = ee.Filter.And(
    ee.Filter.geometry(ee.Geometry.Polygon(verts)),
    ee.Filter.date(start, end),
    )

    # s2 median composite
    s2 = (
      ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filter(lil_filter)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35))
        .map(mask_s2_clouds)
        .median()
    )
    
    # download
    export_params = {
      'image': s2.clip(gee_aoi),
      'description': f'dw_{year}_s2',
      'folder': 'trinity',
      'fileNamePrefix': f'dw_{year}_s2',
      'scale': 10,
      'region': gee_aoi.geometry(),
      'fileFormat': 'GeoTIFF',
      'maxPixels': 1e12 
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    tasks.append(task)


NameError: name 'Path' is not defined


Below is to fix a mistake

In [ ]:
d = Path('/home/michael/data/trinity_trees/sentinel2')
files = d.glob('dw_[0-9][0-9][0-9][0-9]_probability-[0-9]*-[0-9]*.tif')

for f in list(files):
    f.rename('s2'.join(str(f).split('probability')))

In [ ]:
ap2 = geemap.Map(basemap='SATELLITE')
map2.centerObject(gee_aoi, 9) # Centering on the AOI instead of an undefined 'region'
map2.addLayer(gee_aoi, {}, 'AOI')

tasks = []
for year in tqdm(range(2016, 2026)):
    for month in range(12):
        # filter for year round and AOI
        lil_filter = ee.Filter.And(
            ee.Filter.geometry(ee.Geometry.Polygon(verts)),
            ee.Filter.calendarRange(year, year, 'year'),
            ee.Filter.calendarRange(month, month, 'month')
        )

In [ ]:

map2 = geemap.Map(basemap='SATELLITE')
map2.centerObject(gee_aoi, 9) # Centering on the AOI instead of an undefined 'region'
map2.addLayer(gee_aoi, {}, 'AOI')


tasks = []
for year in tqdm(range(2016, 2026)):
  
    # filter for year, month and AOI
    start = ee.Date(f'{year}-01-01')
    end = ee.Date(f'{year}-12-31')
    lil_filter = ee.Filter.And(
    ee.Filter.geometry(ee.Geometry.Polygon(verts)),
    ee.Filter.calendarRange(year, year, 'year'),
    ee.Filter.calendarRange(start, end, 'month')
    )

    # s2 median composite
    s2 = (
      ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filter(lil_filter)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_s2_clouds)
        .median()
    )

    
    ((NIR - Red) / (NIR + Red + L)) * (1 + L)

    # create  mode composite based on highest probable lulc class during of summer months
    classification = dw.select('label');
    dwComposite = classification.reduce(ee.Reducer.mode())
    dw_rgb = dwComposite.visualize(min=0, max=8, palette=dwVisParams['palette']).divide(255)

    # get the most likely class probability (as mean across composite).
    top1_prob = dw.select(
      probabilityBands
      ).reduce(
        ee.Reducer.mean()
        ).reduce(ee.Reducer.max())
    
    # make image for display
    alpha = top1_prob.multiply(0.7).add(0.3)
    dw_rgba = dw_rgb.addBands(alpha.multiply(255))

    # download
    for thing, f'dw_{year}_s2' in [
      
      (s2, f'dw_{year}_s2')
    ]:
      export_params = {
      'image': s2.clip(gee_aoi),
      'description': f'dw_{year}_s2',
      'folder': 'trinity',
      'fileNamePrefix': f'dw_{year}_s2',
      'scale': 10,
      'region': gee_aoi.geometry(),
      'fileFormat': 'GeoTIFF',
      'maxPixels': 1e12 
      }
      task = ee.batch.Export.image.toDrive(**export_params)
      task.start()
      tasks.append(task)

    # clip the composite and add it to the map.
    map.addLayer(dw_rgba.clip(gee_aoi), {}, f'{year} Classified Composite ')


map.add_legend(keys=probabilityBands, colors=dwVisParams['palette'], position='bottomright')

map

100%|██████████| 10/10 [02:18<00:00, 13.89s/it]


Map(center=[40.89654200357109, -122.9134284979662], controls=(WidgetControl(options=['position', 'transparent_…